In [ ]:
#import your libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#your info here
__author__ = "Jared Teerlink"
__email__ = "jteerlink@gmail.com"

In [ ]:
#load the data into a Pandas dataframe
train_features = pd.read_csv('../Data/train_features.csv')
train_salaries = pd.read_csv('../Data/train_salaries.csv')
test_features = pd.read_csv('../Data/test_features.csv')

In [ ]:
#look for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it



train_combined = train_features.merge(train_salaries, on = 'jobId', how = 'left')
train_combined = train_combined[train_combined.salary > 0]
train_combined.head()

In [ ]:
train_combined.groupby(['jobType','degree'])['salary'].mean()

In [ ]:
train_combined[train_combined.salary>100].groupby(train_combined['jobType']).mean()

In [ ]:
train_combined.corr()


In [ ]:
plt.matshow(train_combined.corr())
plt.show()

In [ ]:
numerical_features = ['yearsExperience','milesFromMetropolis']
cat_features = ['jobType','degree','major','industry']

In [ ]:
def cat_bar(x):
    data = x.groupby(x).count()
    plt.bar(data.index,data)
    plt.xticks(rotation=90)
    plt.show()


for i in cat_features:
    cat_bar(train_combined[i])

In [ ]:
train_combined.drop(labels = ['jobId', 'companyId', 'major'],inplace=True,axis=1)

In [ ]:
#Dummy Variables
# major = pd.get_dummies(train_combined['major'], prefix_sep='_', drop_first=True)

# industry = pd.get_dummies(train_combined['industry'], prefix_sep='_', drop_first=True)

# train_combined = pd.concat([train_combined, major, industry], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()

# major_coded = label_enc.fit_transform(train_combined['major'])


industry_coded = label_enc.fit_transform(train_combined['industry'])

# train_combined = pd.concat([train_combined, major_coded, industry_coded], axis=1)

# train_combined['major_enc'] = major_coded
train_combined['industry_enc'] = industry_coded

In [ ]:
degree = [['NONE', 'HIGH_SCHOOL', 'BACHELORS', 'MASTERS', 'DOCTORAL']]
degree_array = np.asarray(train_combined.degree).reshape(-1,1)

from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(categories=degree)
degree_ord = enc.fit_transform(degree_array)

train_combined['degree_ord'] = degree_ord
# ordered by average salary
titles = [['JANITOR', 'JUNIOR', 'SENIOR', 'MANAGER', 'VICE_PRESIDENT', 'CFO', 'CTO', 'CEO']]
titles_array = np.asarray(train_combined.jobType).reshape(-1,1)

enc = OrdinalEncoder(categories=titles)
titles_ord = enc.fit_transform(titles_array)

train_combined['jobType_ord'] = titles_ord

In [ ]:
train_num.head()
print(set(train_num['degree_ord']), set(train_num['jobType_ord']))

# degree 2-4 and type4-7 ==A
# degree 0-1 and type4-7 ==B
# degree 2-4 and type2-3 ==C
# degree 0-1 and type2-3 ==D
# degree 2-4 and type0-1 ==E
# degree 0-1 and type0-1 ==F

def reclassjob(df):
    #some informative note should go here
    if (2 <= df.degree_ord <= 4 and 4 <= df.jobType_ord <= 7):
        return 5
    elif (0 <= df.degree_ord <= 1 and 4 <= df.jobType_ord <= 7):
        return 4
    elif (2 <= df.degree_ord <= 4 and 2 <= df.jobType_ord <= 3):
        return 3
    elif (0 <= df.degree_ord <= 1 and 2 <= df.jobType_ord <= 3):
        return 2
    elif (2 <= df.degree_ord <= 4 and 0 <= df.jobType_ord <= 1):
        return 1
    else:
        return 0
        

In [ ]:
train_num['customclass'] = train_num.apply(reclassjob,axis=1)

In [ ]:
#prepare final data frames

train_num = train_combined.select_dtypes(exclude = 'object')



train_final = train_num.drop(['salary','jobType_ord','degree_ord'],axis=1)

train_salary = train_combined['salary']